## Segmenting and Clustering Neighborhoods in Toronto

In [1]:
# first pip install beautifulsoup4 

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [3]:
# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response=requests.get(wikiurl)
print(response.status_code)

200


In [4]:
# parse data from the html into a beautifulsoup object 
soup = BeautifulSoup(response.text, 'html.parser')
can_postal_table = soup.find('table', {'class':"wikitable"})

In [5]:
# read the HTML table into a list of dataframe object using read_html().

In [6]:
df = pd.read_html(str(can_postal_table))
df = pd.DataFrame(df[0])
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
df[df['Borough'] == 'Not assigned'].value_counts()

Postal Code  Borough       Neighbourhood
M9Z          Not assigned  Not assigned     1
M5Y          Not assigned  Not assigned     1
M3T          Not assigned  Not assigned     1
M3V          Not assigned  Not assigned     1
M3W          Not assigned  Not assigned     1
                                           ..
M7Z          Not assigned  Not assigned     1
M8A          Not assigned  Not assigned     1
M8B          Not assigned  Not assigned     1
M8C          Not assigned  Not assigned     1
M1A          Not assigned  Not assigned     1
Length: 77, dtype: int64

In [8]:
df = df[df.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df.head(11)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
df.shape

(103, 3)